**事前準備 : pip install pyquery**

In [2]:
from pyquery import PyQuery as pq
from collections import OrderedDict
import pandas as pd

In [3]:
# 開爬囉
domain = 'https://www.ptt.cc/bbs/NBA/index5722.html'

q = pq(domain)

In [6]:
# 經過解析後的 q 就是網站的 html DOM Tree
print(q)

<html>
	<head>
		<meta charset="utf-8"/>
		

<meta name="viewport" content="width=device-width, initial-scale=1"/>

<title>看板 NBA 文章列表 - 批踢踢實業坊</title>

<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/bbs-common.css"/>
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/bbs-base.css" media="screen"/>
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/bbs-custom.css"/>
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/pushstream.css" media="screen"/>
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/bbs-print.css" media="print"/>




	</head>
    <body>
		
<div id="topbar-container">
	<div id="topbar" class="bbs-content">
		<a id="logo" href="/">批踢踢實業坊</a>
		<span>›</span>
		<a class="board" href="/bbs/NBA/index.html"><span class="board-label">看板 </span>NBA</a>
		<a class="right small" href="/about.html">關於我們</a>
		<a class="right small" href="/contact.html">聯絡資訊</a>
	</div>
</div>

<

In [4]:
# 獲取本頁所有標題 作者 推文數資訊
title = q('.title a').text()
author = q('.author').text()
push = q('.hl').text()

In [5]:
print(title)

[新聞] 「換血大交易」後5勝4負 騎士防守一樣破 [BOX ] Sixers 110:118 Bucks 數據 [公告] NBA 樂透開獎 [討論] 為何小市場爛隊不簽更多亞洲人 Re: [BOX ] Sixers 110:118 Bucks 數據 [BOX ] Knicks 99:102 Kings 數據 [BOX ] Nets 120:123 Clippers 數據 [新聞] 眾球星祝賀Kobe捧小金人 歐尼爾:我好嫉妒 [專欄] 從DeRozan的坦白看職業運動不能說的秘密 [新聞] 10年來第一次 暴龍例行賽橫掃黃蜂 [新聞] 同時擁有冠軍戒與小金人 Kobe史上第一人 [公告] NBA 板 開始舉辦樂透! [新聞] Kobe Bryant奧斯卡獲獎 不忘諷刺福斯新聞 Re: [討論] 為何小市場爛隊不簽更多亞洲人 [花邊] LBJ祝賀Kobe：向Kobe贏得奧斯卡致敬 Re: [討論] 為何小市場爛隊不簽更多亞洲人 Re: [討論] 為何小市場爛隊不簽更多亞洲人 [花邊] Bleacher Report推特發佈Kobe得獎表情集 [專欄] 錯過柯瑞衰敗多年 尼克這次該鎖定楊恩


**很明顯這樣的資訊並不是我們要的，因為所有文章的訊息都擠在一起了，所以接下來我們要針對每一篇文章分別爬資訊**

In [10]:
info = []
for article in q('.r-ent'):
    pass
print('如果沒有重新解析:',article)
print('重新解析後:',pq(article))

如果沒有重新解析: <Element div at 0x9411408>
重新解析後: <div class="r-ent">
			<div class="nrec"><span class="hl f3">44</span></div>
			<div class="mark"/>
			<div class="title">
			
				<a href="/bbs/NBA/M.1520232042.A.053.html">[專欄] 錯過柯瑞衰敗多年 尼克這次該鎖定楊恩</a>
			
			</div>
			<div class="meta">
				<div class="date"> 3/05</div>
				<div class="author">asdf1256</div>
			</div>
		</div>

		
        
	


In [11]:
# 獲取每篇文章分別的標題 作者 推文數資訊
info = []
for article in q('.r-ent'):
    info_dict = OrderedDict()

    # 每一篇文章的資訊分別塞到字典裡頭
    article_q = pq(article) # 記得要重新用 pq再解析一次
    info_dict['title'] = article_q('.title a').text()
    info_dict['author'] = article_q('.author').text()
    info_dict['push'] = article_q('.hl').text()

    # 將字典存入list
    info.append(info_dict)

In [12]:
# 將list轉成pandas dataframe
data = pd.DataFrame(info)
# 存檔
data.to_csv('ptt_nba_info.csv', encoding='utf-8', index=None)

In [13]:
data.head()

,title,author,push
0,[新聞] 「換血大交易」後5勝4負 騎士防守一樣破,Wojnarowski,46
1,[BOX ] Sixers 110:118 Bucks 數據,Rambo,37
2,[公告] NBA 樂透開獎,[彩券],18
3,[討論] 為何小市場爛隊不簽更多亞洲人,siriusc,56
4,Re: [BOX ] Sixers 110:118 Bucks 數據,spurs2120,16


**現在已經有了每篇文章個別的標題 作者 推文數資訊，那如果想知道每篇文章的推文該怎麼做呢?**

In [18]:
# 獲取每篇文章的url
url_list = []
for node in q('.title a'):
    url = 'https://www.ptt.cc' + str(pq(node).attr('href')) # 記得要重新用 pq再解析一次
    url_list.append(url)

In [19]:
# 進入每篇文章的 url 獲得詳細資訊
page_list = []
for page in url_list:
    page_dict = OrderedDict()

    page_q = pq(page)
    page_dict['author'] = page_q('.article-metaline:nth-child(1) .article-meta-value').text()
    page_dict['push_content'] = page_q('.push-content').text()

    page_list.append(page_dict)

In [20]:
page_data = pd.DataFrame(page_list)
page_data.to_csv('ptt_nba_pageinfo.csv', encoding='utf-8', index=None)

In [21]:
page_data.head()

,author,push_content
0,Wojnarowski (@wojespn),: 22 = 14 + 6? : 反正躺著進東去季後賽 還可以慢慢磨合 : 30->18 我...
1,,: 頭 : 唉 失誤 : 我鹿逆轉幫我贏錢！！ : 76人認真點好嗎 : 公鹿打成這樣季後賽...
2,[彩券],: 雙星54分小牛輸 故開二公鹿贏~ : 錢 : 推 版主辛苦了 : 版主帶大家賺錢 : E...
3,siriusc (慟!甲甲),: 等亞洲老闆買阿 不是有人想買灰熊什麼的? : 楊能替補1分鐘都要偷笑了 當NBA這麼缺人...
4,spurs2120 (Fundamental),": 兩度? 是指 落後15分以上 然後追平 接著又落後 : 3年前的76人還在坦 應該不是""..."
